<a href="https://colab.research.google.com/github/polyu-llt/fin-sharedtask/blob/master/classifier5_tfidf_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from nltk import pos_tag, word_tokenize #Run `python3 -m nltk.downloader averaged_perceptron_tagger` before


def tag_text(words_set):
    postag_column = pos_tag(words_set)
    #print (postag_column)
    words, pos = zip(*postag_column) #(from https://stackoverflow.com/questions/42003418/getting-a-list-of-pos-tags-only-from-the-tuples-of-word-pos-tag)
    print (len(pos))
    return(pos)

In [0]:


def make_sets(file):
    Y_set = []
    X_set = []

    test = list(open(file, "r", encoding='utf-8').readlines())
    test = [s.strip() for s in test]
    all_set = [x.replace('_Enter', '') for x in test]


    for line in all_set:    
        if '_END' in line:
            Y_set.append('END')
        elif '_BEGIN' in line:
            Y_set.append('BEGIN')
        else:
            Y_set.append('_')

    X_set = [x.replace('_END', '').replace('_BEGIN','') for x in all_set]
    #X_tags = tag_text(X_set)
    
    print ("length of X: ", len(X_set), "length of Y: ", len(Y_set))
    return(X_set, Y_set)




In [6]:

from google.colab import files
uploaded = files.upload()

Saving Dev_en_new_labelled.txt to Dev_en_new_labelled.txt


In [8]:
test_file = "Dev_en_new_labelled.txt"
train_file = "Train_en_new_labelled.txt"
#file_tags_dev = "Dev_tags.txt"
#X_tags_dev = list(open(file_tags_dev, "r", encoding='utf-8').readlines())
#X_tags_dev = [s.strip() for s in X_tags_dev]

X_test,Y_test = make_sets(test_file)
#print (X_tags_dev[:100])
X_train, Y_train = make_sets(train_file)

length of X:  49859 length of Y:  49859
length of X:  904057 length of Y:  904057


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

all_X = X_train + X_test
tfidf_vect = TfidfVectorizer()#analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(all_X)

xtrain_tfidf =  tfidf_vect.transform(X_train)
xvalid_tfidf =  tfidf_vect.transform(X_test)

print(xtrain_tfidf.shape)
print(xvalid_tfidf.shape)
print(xtrain_tfidf[:10])



(904057, 9719)
(49859, 9719)
  (0, 5106)	1.0
  (1, 4314)	1.0
  (3, 8310)	1.0
  (4, 9421)	1.0
  (5, 1824)	1.0
  (7, 7013)	1.0
  (8, 7287)	1.0


In [0]:
print (xtrain_tfidf[:100])

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
print('import done')


Using TensorFlow backend.


import done


In [0]:
from keras.utils import np_utils
le = LabelEncoder()
Y_test = le.fit_transform(Y_test)
Y_test_cat = np_utils.to_categorical(Y_test, 3)
Y_train = le.fit_transform(Y_train)
Y_train_cat = np_utils.to_categorical(Y_train, 3)
#X_tags = le.fit_transform(X_tags_dev)

print (Y_train_cat[:10])

In [0]:
def get_simple_model():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(9719,)))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(3, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc',metrics.binary_accuracy])
    print('compile done')
    return model

  
def check_model(model,x_train,y_train,x_val,y_val):
    model.fit(x_train,y_train,batch_size=32,epochs=5,verbose=1,validation_data=(x_val, y_val))
    
m = get_simple_model()
check_model(m,xtrain_tfidf,Y_train_cat, xvalid_tfidf, Y_test_cat)

compile done
Instructions for updating:
Use tf.cast instead.
Train on 904057 samples, validate on 49859 samples
Epoch 1/5
488896/904057 [===============>..............] - ETA: 20:46 - loss: 0.1786 - acc: 0.9503 - binary_accuracy: 0.9666

**Code not tested not to execute yet:**

In [0]:
y_train_predclass = m.predict_classes(x_train_tfidf ,batch_size=batch_size)
y_test_predclass = m.predict_classes(x_valid_tfidf,batch_size=batch_size)
from sklearn.metrics import accuracy_score,classification_report

print ("nnDeep Neural Network - Train accuracy:"),(round(accuracy_score( Y_train_cat, y_train_predclass),3))
print ("nDeep Neural Network - Test accuracy:"),(round(accuracy_score( Y_test_cat,y_test_predclass),3))
print ("nDeep Neural Network - Train Classification Report")
print (classification_report(Y_train_cat,y_train_predclass))
print ("nDeep Neural Network - Test Classification Report")
print (classification_report(Y_test_cat,y_test_predclass))


X_to_predict = X_tags[:200]
Y_test_last200 = Y_test[:200]
print (X_to_predict)
predictions = m.predict(X_to_predict)
#print (Y_test_cat[:200])
print ("Predictions: ", predictions)

[19 19 19 19 14 18 19 18 14  9 18 19  6 19  6 19  5 19  7  9 19 19 19  9
 19 13 10 21 18 37 26 14 13 24 37 35 13 10 18 13 10 16 14 14 18 13 19 19
 19  3 10 18 19 19  4 34 10 14 18  7 18  8 10 18 13 10 16 18  7 14 18  5
 13 35 13 14 14 18  6 10 21 18 13 10 19  3 10 18 19 19  4  8 10 14 14  8
 14  7 14 21 17 32 35 14 13 18 13 10 34 21  8 21 21  6 24 37 35 31 32 18
 13 10 18 38 37 26 35 13 10 19  5 10 21 35 26  5 10 16 14 18  8 10 14 18
  7 14 18  6 10 14 18 13 10 19 37 34  6 19 19 19 19  3 10 18 19 19  4 37
 10 14 18 18 19 18 18  3 14 19 19  4 33 13 19 24 13 10 19 18 13  9 19  9
 13 21 13 14 18  3 10 18]
Predictions:  [[1.6342312e-02 0.0000000e+00 9.8387313e-01]
 [1.6342312e-02 0.0000000e+00 9.8387313e-01]
 [1.6342312e-02 0.0000000e+00 9.8387313e-01]
 [1.6342312e-02 0.0000000e+00 9.8387313e-01]
 [3.3486545e-02 0.0000000e+00 9.6524340e-01]
 [1.7812610e-02 0.0000000e+00 9.8236084e-01]
 [1.6342312e-02 0.0000000e+00 9.8387313e-01]
 [1.7812610e-02 0.0000000e+00 9.8236084e-01]
 [3.3486545e-02 